## FRISS with MFC

Implementation of the FRISS using the Media Frames Corpus (MFC) from Card et al. (2015). 

In [1]:
!pip install nltk

In [2]:
import nltk
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [3]:
import os

os.listdir(os.getcwd())

['.gradient',
 'README.md',
 'notebooks',
 '.git',
 'assets',
 'setup.sh',
 '.ipynb_checkpoints',
 'data',
 '.gitignore',
 'datasets']

In [4]:
labeled_path = "data/mfc/immigration_labeled.json"
unlabeld_path = "data/mfc/immigration_unlabeled.json"
codes_path = "data/mfc/codes.json"

In [5]:
# load data from path 
import json

with open(labeled_path) as f:
    labeled = json.load(f)

with open(unlabeld_path) as f:
    unlabeld = json.load(f)

with open(codes_path) as f:
    codes = json.load(f)

In [6]:
import pandas as pd
from nltk.tokenize import sent_tokenize

In [7]:
def get_labeled_data(labeled, codes):
    # articles list
    articles_list = []

    # Iterate through the data to fill the DataFrame
    for article_id, article_data in labeled.items():
        annotations_data = article_data['annotations']

        irrelevant_dict = annotations_data['irrelevant']

        text = article_data['text']
        irrelevant = article_data['irrelevant']

        # if primary_frame is none set to 15.0
        if article_data['primary_frame'] is not None:
            primary_frame = str(article_data['primary_frame']).split(".")[0] + ".0"
        else:
            primary_frame = "15.0"

        # get primary frame from code
        primary_frame = str(codes[primary_frame])

        # split text into sentences using nltk library
        sentences = sent_tokenize(text)

        # iterate through sentences
        for sentence in sentences:
            article = {
                'article_id': article_id,
                'irrelevant': irrelevant,
                'text': sentence,
                'document_frame': primary_frame
            }

            articles_list.append(article)

    # Create a DataFrame to store the results
    df = pd.DataFrame(articles_list, columns=['article_id', 'irrelevant', 'text', 'document_frame'])

    return df

In [8]:
def get_unlabeled_data(unlabeled):
    # articles list
    articles_list = []

    for idx, article in enumerate(unlabeled):
        article_id = f"unlabeled_{idx}" 
        text = article['text']

        # split text into sentences using nltk library
        sentences = sent_tokenize(text)

        # iterate through sentences
        for sentence in sentences:
            article = {
                'article_id': article_id,
                'text': sentence
            }

            articles_list.append(article)

    # Create a DataFrame to store the results
    df = pd.DataFrame(articles_list, columns=['article_id', 'text'])

    return df

## Get labeled and unlabeled data

In [9]:
df_labeled = get_labeled_data(labeled, codes)
df_unlabeled = get_unlabeled_data(unlabeld)

print("Labeled Count: ", len(df_labeled))
print("Unlabeled Count: ", len(df_unlabeled))

Labeled Count:  74468
Unlabeled Count:  460535


In [10]:
def preprocess_labeled_df(df):
    df = df[df["irrelevant"] == False][["article_id", "text", "document_frame"]]

    # create for each code a col and fill with 1 if code is in code col
    df = pd.concat([df, pd.get_dummies(df['document_frame'])], axis=1)  

    return df

In [11]:
df_labeled = preprocess_labeled_df(df_labeled)

In [12]:
df_labeled.head()

,article_id,text,document_frame,Capacity and Resources,Crime and Punishment,Cultural Identity,Economic,External Regulation and Reputation,Fairness and Equality,Health and Safety,"Legality, Constitutionality, Jurisdiction",Morality,Other,Policy Prescription and Evaluation,Political,Public Sentiment,Quality of Life,Security and Defense
0,Immigration1.0-10005,IMM-10005\n\nPRIMARY\n\nImmigrants without HOP...,Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Immigration1.0-10005,It mounted as students went around the room te...,Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,Immigration1.0-10005,Georgia Tech.,Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Immigration1.0-10005,University of Georgia.,Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,Immigration1.0-10005,"""All I could say was, 'I'm planning to see if ...",Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [13]:
df_unlabeled.head()

,article_id,text
0,unlabeled_0,IMM-10000\n\nPRIMARY\n\nMetro Briefing New Yor...
1,unlabeled_0,"As part of the scheme, Ms. Holzer convinced Sp..."
2,unlabeled_0,Katherine E. Finkelstein (NYT)
3,unlabeled_1,IMM-10003\n\nPRIMARY\n\nAmnesty Works for Amer...
4,unlabeled_1,All working families would benefit from immigr...


In [14]:
df_labeled.shape, df_unlabeled.shape

((67480, 18), (460535, 2))

# Fine tune

In [15]:
from transformers import RobertaTokenizer, RobertaForMaskedLM, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

In [16]:
class ArticlesDataset(Dataset):
    def __init__(self, articles, tokenizer):
        self.encodings = tokenizer(articles, max_length=512, truncation=True, padding='max_length', return_tensors='pt')

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)


In [17]:
# combine labeled and unlabeled data
df = pd.concat([df_labeled, df_unlabeled])

df.shape

(528015, 18)

In [18]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# create train and test dataset
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df["text"].tolist(), test_size=0.2, random_state=42)

train_dataset = ArticlesDataset(train_df, tokenizer)
test_dataset = ArticlesDataset(test_df, tokenizer)


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [20]:
from torch.utils.data import DataLoader
from transformers import BertTokenizer, DataCollatorForLanguageModeling

# Data collator for MLM
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=data_collator)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=data_collator)


In [21]:
# Load model
model = RobertaForMaskedLM.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

In [22]:
from transformers import AdamW

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
device

device(type='cuda')

# Train model

In [25]:
from transformers import RobertaForMaskedLM, AdamW
from tqdm.notebook import tqdm
import os
from torch.utils.data import DataLoader
from pathlib import Path

In [26]:
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [27]:
# check if model is at gpu
next(model.parameters()).is_cuda

True

In [28]:
# Training loop with tqdm
def train(epoch, model, train_loader, optimizer, device):
    model.train()
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        inputs = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

# Testing loop with tqdm
def test(model, test_loader, device):
    model.eval()
    total_loss = 0
    loop = tqdm(test_loader, leave=False)
    for batch in loop:
        inputs = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(inputs, labels=labels)
        loss = outputs.loss.item()
        total_loss += loss
        loop.set_postfix(test_loss=loss)
    avg_loss = total_loss / len(test_loader)
    loop.set_postfix(avg_test_loss=avg_loss)


In [29]:
# Define the path for model saving
model_save_path = 'models/roberta-finetuned'
num_epochs = 3 # Set the number of epochs

for epoch in range(1, num_epochs + 1):
    train(epoch, model, train_loader, optimizer, device)
    test(model, test_loader, device)
    
    # Save after each epoch
    model_save_name = f'roberta-base-epoch-{epoch}.pt'
    model_save_full_path = os.path.join(model_save_path, model_save_name)
    
    # Ensure directory exists
    Path(model_save_path).mkdir(parents=True, exist_ok=True)
    
    # Save the model
    torch.save(model.state_dict(), model_save_full_path)
    
    # If there is a model from the previous epoch, delete it to save space
    previous_model_path = os.path.join(model_save_path, f'roberta-base-epoch-{epoch-1}.pt')
    if os.path.isfile(previous_model_path):
        os.remove(previous_model_path)
    print(f"Saved model for epoch {epoch} and removed previous model.")

  0%|          | 0/13201 [00:00<?, ?it/s]

/tmp/ipykernel_335/880483901.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


KeyboardInterrupt: 